![Clase aprendizaje no supervisado](https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/S5_sistemas_recomendacion_I/main/figs/taller-meca-aprendizaje%20no%20supervisado_banner%201169%20x%20200%20px%20-05.png)

# Filtrado Colaborativo Basado en Usuarios.

Este *cuaderno* trata sobre filtrado colaborativo basado en usuarios. El objetivo del *cuaderno* es que usted obtenga una visión general del problema predictivo de los sistemas de recomendación que utilizan filtrado colaborativo basado en usuario, aprenda distintos algoritmos que lo implementan, y que sea capaz de reconocer sus características, funcionamiento, y  desarrollarlos en `Python`.

**NO** es necesario editar el archivo o hacer una entrega. Sin embargo, los ejemplos contienen celdas con código ejecutable (`en gris`), que podrá modificar  libremente. Esta puede ser una buena forma de aprender nuevas funcionalidades del *cuaderno*, o experimentar variaciones en los códigos de ejemplo.

## Introducción: el problema de predicción

Para entender un poco mejor cuál es el problema al que nos enfrentamos, supongamos que tenemos una matriz con $m$ usuarios y $n$ ítems. En esta matriz cada fila representa un usuario y cada columna un ítem. El valor de la celda denota el rating que le dió cada usuario al ítem. Este valor lo denotamos como $r_{ij}$ que será entonces el rating que le dio el usuario $i$ al ítem $j$. Un ejemplo de esta matriz sería la siguiente:

![Clase aprendizaje no supervisado](https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/S5_sistemas_recomendacion_I/main/figs/fig1_colab.png)

Esta matriz tiene 7 usuarios ($m=7$) y 6 ítems ($n=6$). El usuario 1 le otorgó al ítem 1 un rating de 4, por lo tanto $r_{11}=4$. Sin embargo, no todos los usuarios calificaron/utilizaron todos los ítems y por lo tanto no todos tienen ranking. En el caso del usuario 1 este no utilizó los ítems 3, 4, y 6; por lo tanto aparecen con un signo de interrogación. Estos datos están faltando en la matriz.


Consideremos otro ejemplo más concreto. Imaginemos que somos una compañía como Netflix y tenemos un repositorio de 20.000 películas y 5.000 usuarios. Tenemos además un sistema que registra la calificación que cada usuario le otorga a una película en particular. Es decir, tenemos una  matriz de tamaño 5,000 × 20,000. Sin embargo, es muy probable que los usuarios sólo habrán visto sólo una fracción de las películas (difícilmente todos vieron 20.000 películas!). Por lo tanto, la matriz será poco densa, la mayoría de las entradas en la matriz estarán vacías.


El problema de predicción, tiene por objetivo predecir estos valores faltantes utilizando toda la información que tenemos a disposición: las calificaciones registradas, datos sobre películas, datos sobre usuarios, etc. Si el sistema es capaz de predecir con precisión los valores que faltan, podrá dar excelentes recomendaciones. Por ejemplo, si el usuario $i$ no ha utilizado el ítem $j$, pero nuestro sistema predice una calificación muy alta, es entonces muy probable que le guste $j$ si lo descubre a través del sistema de recomendación.

Para ilustrar cómo construir e implementar este tipo de sistema de recomendación utilizaremos nuevamente los datos de  [MovieLens](https://grouplens.org/datasets/movielens/latest/) provista abiertamente por [grouplens](https://grouplens.org/about/what-is-grouplens/) para: **"avanzar la teoría y la práctica de la computación social mediante la construcción y la comprensión de sistemas *(de recomendación)* utilizados por personas reales".**

Esta versión de los datos contiene varias bases con información de los usuarios, de las películas, y de los ratings. Tendremos que combinar estas bases para obtener una matriz similar a la ilustrada en la gráfica anterior. Carguemos entonces las librerías y los datos de usuarios:

In [2]:
# Cargamos las librerías a utilizar
import pandas as pd
import numpy as np
# Cargamos los datos de los usuarios
u_cols = ['user_id', 'edad', 'genero', 'ocupacion', 'codigo_postal']

users = pd.read_csv('https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/S5_sistemas_recomendacion_I/main/data/u.user',
                    sep='|', names=u_cols,
                    encoding='latin-1')
users.head()

,user_id,edad,genero,ocupacion,codigo_postal
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


Podemos ver que esta base contiene un identificador de usuario, su edad, género, ocupación, y el código postal donde viven. Luego cargamos la base de películas:

In [3]:
# Cargamos los datos de las películas
i_cols = ['movie_id','titulo', 'fecha_estreno', 'fecha_estreno_video', 'URL_IMDb', 'desconocido', 'Accion', 'Aventura',
  'Animacion', 'Infantil', 'Comedia', 'Crimen', 'Documental', 'Drama', 'Fantasia',
  'Cine-Noir', 'Horror', 'Musical', 'Misterio', 'Romance', 'Ciencia_ficcipn', 'Thriller', 'Guerra', 'Western']

movies = pd.read_csv('https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/S5_sistemas_recomendacion_I/main/data/u.item',
                     sep='|', names=i_cols, encoding='latin-1')
movies.head()

,movie_id,titulo,fecha_estreno,fecha_estreno_video,URL_IMDb,desconocido,Accion,Aventura,Animacion,Infantil,...,Fantasia,Cine-Noir,Horror,Musical,Misterio,Romance,Ciencia_ficcipn,Thriller,Guerra,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


Esta base contiene un identificador de película, el título, cuándo fue estrenada en el cine, cuándo fue estrenada en video, la dirección web a IMDb, y el género al que pertenece, incluyendo una columna que marca como desconocido si el género no pertenece a ninguna de las restantes. Notemos además que una película puede pertenecer a varios géneros, por ejemplo: Toy Story pertenece  al genero Animación e Infantil. Pero por ahora nos importa sólo el identificador y el título de la película, por lo que nos quedamos con estas columnas:

In [4]:
movies = movies[['movie_id', 'titulo']]
movies.head()

,movie_id,titulo
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


La última base que necesitamos cargar es la de ratings:

In [6]:
# Cargamos los datos de los ratings
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']

ratings = pd.read_csv('https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/S5_sistemas_recomendacion_I/main/data/u.data',
                      sep='\t', names=r_cols,
                      encoding='latin-1')
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


Esta base tiene información sobre qué `rating` el usuario (`user_id`) le otorgó a cada película (`movie_id`), e información de cuándo fue que hizo tal calificación (`timestamp`). Esta última columna no la utilizaremos por lo que la excluiremos de la base.

In [7]:
# Quitamos la columna timestamp
ratings = ratings.drop('timestamp', axis=1)
ratings.head()

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


Con estos elementos podemos construir la matriz que vincule usuarios, películas y ratings. Para ello usamos la función `pivot_table`.

In [8]:
r_matrix = ratings.pivot_table(values='rating', index='user_id', columns='movie_id')

r_matrix.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
r_matrix.shape

(943, 1682)

Esta matriz entonces vincula los usuarios con las películas. Las filas denotan los 943 usuarios y las columnas las 1682 películas en la base. El usuario 1 le otorgó a la película 1 un rating de 5, por lo tanto $r_{11}=5$. Al igual que en el ejemplo anterior, no todos los usuarios calificaron/vieron todas las películas y por lo tanto no todas tienen ranking. En el caso del usuario 1 este no calificó las películas 1673 a 1682 y por lo tanto aparecen con `NaN`; estos datos están faltando en la matriz.

La tarea es entonces buscar estrategias para completar estos datos faltantes.

## Filtrado colaborativo sencillo: medias, medias ponderadas, y datos demográficos.

### Medias

Comencemos por la estrategia quizás más simple e intuitiva para completar las celdas faltantes. Esta estrategia consiste en calcular el rating promedio que le asignó cada usuario que calificaron esta película. No hacemos distinción entre los usuarios, y el rating de cada uno recibirá el mismo peso. Habrá casos donde ninguna de las películas ha sido calificada, en tales situaciones le pondremos un rating default de 3.

Para evaluar además la performance de esta estrategia, dividiremos la base original en bases de entrenamiento y prueba.

In [10]:
# Importamos la función train_test_split
from sklearn.model_selection import train_test_split

#Asignamos `X` como la base original de ratings e `y` el usuario

X = ratings.copy()
y = ratings['user_id']

# Partimos la base en entrenamiento y prueba estratificando por usuario
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, stratify=y, random_state=42)

y definiremos como métrica el error cuadrático medio $\sqrt{(\frac{1}{n}\sum_{i=1}^n(y_i-\hat{y}_i)^2)}$:

In [11]:
# Importamos la función mean_squared_error
from sklearn.metrics import mean_squared_error

# Creamos una funcion que calcula la raíz del error cuadrático medio (RMSE)
def rmse(y_true, y_pred):
       return np.sqrt(mean_squared_error(y_true, y_pred))

Con la base lista podemos construir nuestra función recomendadora:

In [12]:
def cf_user_mean(user_id, movie_id):

    # Primero verificamos si la película está en la matriz
    if movie_id in r_matrix:
        # Si esta calculamos la media de los ratings
        mean_rating = r_matrix[movie_id].mean()

    else:
        # Si no lo está, asignamos el valor de 3 (los invito a que prueben con otros valores)
        mean_rating = 3.0

    return mean_rating

Evaluemos ahora el RMSE en la base de prueba, pero primero construyamos la función de evaluación:

In [13]:
def rmse_recom(cf_model):
    # Construimos una lista con las tuplas usuario-película en la base de entrenamiento
    id_pairs = zip(X_test['user_id'], X_test['movie_id'])
    # Predecimos el rating para cada tupla usuario-película
    y_pred = np.array([cf_model(user, movie) for (user, movie) in
   id_pairs])
    # Extraemos los ratings que dieron los usuarios en la base de prueba
    y_true = np.array(X_test['rating'])
    # Retornamos el RMSE
    return rmse(y_true, y_pred)


Con la función obtenemos nuestra medida de performance:

In [14]:
rmse_recom(cf_user_mean)

np.float64(1.0062161517166315)

Este será nuestra medida base de desempeño y buscaremos mejorar este desempeño en los  modelos subsiguientes.

### Medias ponderadas

En el modelo anterior, asignamos el mismo peso a todos los usuarios. Sin embargo, podríamos pensar que obtendríamos mejores recomendaciones si estas se generan a partir de usuarios similares.

En esta sección generaremos recomendaciones dando mayor peso a usuarios cuyas calificaciones son más similares a las del usuario que queremos generar la recomendación.

Esta estrategia sugiere que deberíamos ponderar más ciertos rankings. La pregunta natural que surge es: ¿Cómo construimos estos pesos? En este caso construiremos el peso basado en la similitud de las calificaciones. Matemáticamente:

$$
r_{um}=\frac{\sum_{u',u'\neq u}sim(u,u').r_{u'm}}{\sum_{u',u'\neq u}sim(u,u')}
$$

donde $sim(u,u')$ es una función que nos retorna la similitud entre las calificaciones del usuario $u$ y el usuario $u'$.  Es decir, la predicción del rating del usuario $u$ para la película $m$, $r_{um}$, es la suma ponderada de los ratings de los otros usuarios ($u'$) a esta película, ponderado por cuán similares son los usuarios $u'$ a $u$.


#### Distancia de coseno

Es necesario entonces definir la función que mide la similitud entre usuarios $sim(u,u')$.  Existen múltiples medidas de distancia que se utilizan para medir la similitud. En el *cuaderno: Introducción al Análisis de Clusters* describimos varias de estas medidas que también pueden ser utilizadas en este contexto. Sin embargo, la distancia de coseno suele ser la más utilizada en los sistemas de recomendación.

Matemáticamente esta se define como:

$$
coseno(x,y)=\frac{x.y'}{|x||y|}
$$

Es decir, es el cociente del producto punto, dividido por las normas de los vectores. Esto hace que la medida esté entre -1 y 1. Geométricamente esta medida es el coseno entre el ángulo de los dos vectores en el espacio. En la figura a continuación la medida de distancia estará entonces definida por el coseno del ángulo $\theta$:


![Clase aprendizaje no supervisado](https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/S5_sistemas_recomendacion_I/main/figs/dist_cos.png)

Cuando el coseno es igual a 1 o el ángulo es 0, los vectores son exactamente similares. Por el otro, si el coseno da -1, el ángulo es de 180 grados, esto nos denota que dos vectores son exactamente disimilares.

Notemos que, si dos vectores, $x$ e $y$, tienen media cero, el coseno del angulo entre ambos coincidirá con el coeficiente de correlación de Pearson que estudiamos en el *cuaderno: Introducción al Análisis de Clusters*.

La distancia del coseno suele ser una medida de similitud ampliamente utilizada en los sistemas de recomendación por varias razones:

   - *Independencia de magnitud*: La distancia del coseno mide la similitud entre dos vectores basándose en el ángulo entre ellos, en lugar de depender de las magnitudes de los vectores. Esto es útil en los sistemas de recomendación, donde las magnitudes de las características (por ejemplo, la calificación de un usuario para un artículo) pueden variar significativamente entre diferentes usuarios o elementos. Al ser independiente de la magnitud, el coseno permite capturar patrones de similitud más robustos.

   - *Espacios de alta dimensión*: En los sistemas de recomendación, las características que representan usuarios o elementos a menudo se representan como vectores en espacios de alta dimensión. En estos espacios, la noción de distancia euclidiana puede volverse menos significativa debido al fenómeno conocido como "maldición de la dimensionalidad". La distancia del coseno puede ser más adecuada en estos casos porque se centra en la orientación relativa de los vectores, lo que permite medir la similitud de manera más efectiva.

   - *Eficiencia computacional*: Calcular la distancia del coseno es computacionalmente eficiente en comparación con otras métricas de similitud más complejas, como la distancia de Mahalanobis o la distancia euclidiana en espacios de alta dimensión. Esto es importante en sistemas de recomendación que manejan grandes conjuntos de datos y deben realizar cálculos rápidos para generar recomendaciones en tiempo real.

   - *Contexto de recomendación*: En muchos sistemas de recomendación, las preferencias de los usuarios pueden estar dispersas, lo que significa que solo han interactuado con una pequeña parte del conjunto total de elementos. La distancia del coseno tiende a funcionar bien en estos escenarios, ya que ignora las diferencias entre características no observadas, lo que puede conducir a una mejor generalización y recomendaciones más precisas.

A pesar de estas ventajas, es importante destacar que no existe un enfoque único para todos los sistemas de recomendación. La elección de la medida de similitud depende en gran medida del contexto y la naturaleza de los datos en un sistema específico. Por ejemplo,

 - En caso que las magnitudes sean importantes, la distancia euclidiana es una medida apropiada para utilizar.
 - Si la correlación es lo que importa, entonces la correlación de Pearson o el coseno son más apropiadas.
 - Si la popularidad del ítem bajo estudio es importante, entonces el producto escalar será una medida adecuada. Puesto que si los ítems aparecen con mucha frecuencia (por ejemplo, videos populares de YouTube) estos tienden a normas grandes y el producto escalar capturará mejor esta información. Sin embargo, si no somos cuidadosos, aquellos artículos populares terminarán siendo los más recomendados. En casos como este, se puede definir una medida que "regularice" las normas, como por ejemplo: $similitud(x,y)=|x|^\alpha  |y|^\alpha cos(x,y)$ para algún $\alpha\in(0,1)$.

#### Implementación
Completado este breve desvío, estamos en condiciones de implementar nuestro recomendador basado en medias ponderadas. Comenzamos primero imputando 0 a los valores faltantes. Esto lo hacemos para poder calcular la similitud de coseno.

In [15]:
# Rellenamos los faltantes con 0
r_matrix_dummy = r_matrix.copy().fillna(0)

Con esto estamos en condiciones de usar la función `cosine_similarity` que va a calcular la similitud de coseno.

In [16]:
# Importamos cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity

#Calculamos la similitud de coseno
cosine_sim = cosine_similarity(r_matrix_dummy, r_matrix_dummy)

# Transformamos la matriz resultante en un dataframe
cosine_sim = pd.DataFrame(cosine_sim, index=r_matrix.index, columns=r_matrix.index)

cosine_sim.head(10)

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.166931,0.047460,0.064358,0.378475,0.430239,0.440367,0.319072,0.078138,0.376544,...,0.369527,0.119482,0.274876,0.189705,0.197326,0.118095,0.314072,0.148617,0.179508,0.398175
2,0.166931,1.000000,0.110591,0.178121,0.072979,0.245843,0.107328,0.103344,0.161048,0.159862,...,0.156986,0.307942,0.358789,0.424046,0.319889,0.228583,0.226790,0.161485,0.172268,0.105798
3,0.047460,0.110591,1.000000,0.344151,0.021245,0.072415,0.066137,0.083060,0.061040,0.065151,...,0.031875,0.042753,0.163829,0.069038,0.124245,0.026271,0.161890,0.101243,0.133416,0.026556
4,0.064358,0.178121,0.344151,1.000000,0.031804,0.068044,0.091230,0.188060,0.101284,0.060859,...,0.052107,0.036784,0.133115,0.193471,0.146058,0.030138,0.196858,0.152041,0.170086,0.058752
5,0.378475,0.072979,0.021245,0.031804,1.000000,0.237286,0.373600,0.248930,0.056847,0.201427,...,0.338794,0.080580,0.094924,0.079779,0.148607,0.071459,0.239955,0.139595,0.152497,0.313941
6,0.430239,0.245843,0.072415,0.068044,0.237286,1.000000,0.489255,0.201369,0.183951,0.551713,...,0.385838,0.111828,0.190075,0.225142,0.137901,0.111852,0.352449,0.144446,0.317328,0.276042
7,0.440367,0.107328,0.066137,0.091230,0.373600,0.489255,1.000000,0.284951,0.145650,0.487024,...,0.456183,0.114179,0.112422,0.117860,0.153353,0.107027,0.329925,0.059993,0.282003,0.394364
8,0.319072,0.103344,0.083060,0.188060,0.248930,0.201369,0.284951,1.000000,0.085942,0.233289,...,0.239171,0.067626,0.094126,0.096483,0.169737,0.095898,0.246883,0.146145,0.175322,0.299809
9,0.078138,0.161048,0.061040,0.101284,0.056847,0.183951,0.145650,0.085942,1.000000,0.198223,...,0.082199,0.048640,0.163049,0.131415,0.118232,0.039852,0.120495,0.143245,0.092497,0.075617


Tenemos entonces como resultado una matriz que muestra la similitud de coseno entre los diferentes usuarios. La matriz diagonal que muestra $1$ nos dice que la similitud entre el usuario 1 y sí mismo es 1. Note que el 0.16 dice que la similitud entre 1 y 2 es de 0.16. En este caso, dado que estamos evaluando ratings positivos, esta matriz retornará entradas positivas.

Con esta matriz calculada podemos entonces utilizarla para construir nuestra función recomendadora basada en medias ponderadas.

In [17]:
def cf_user_wmean(user_id, movie_id):

    # Primero verificamos si la película esta en la matriz
    if movie_id in r_matrix:

        #Buscamos las medidas de similitud con los otros usuarios
        sim_scores = cosine_sim[user_id]


        # Obtenemos los ratings no faltantes de la matriz bajo evaluación
        m_ratings = r_matrix[movie_id]

         # Obtenemos los índicies de las películas sin rating
        idx = m_ratings[m_ratings.isnull()].index

        # Nos quedamos con similitudes y ratings completos
        sim_scores = sim_scores.drop(idx)
        m_ratings = m_ratings.dropna()

        # Calculamos la media ponderada
        wmean_rating = np.dot(sim_scores, m_ratings)/ sim_scores.sum()

    else:
        # Si no tenemos ninguna información retornamos 3
        wmean_rating = 3.0

    return wmean_rating

Estamos en condiciones entonces de evaluar este modelo con la función antes creada:

In [18]:
rmse_recom(cf_user_wmean)

np.float64(0.964396004447058)

Notemos que esta simple "sofisticación" mejora el desempeño predictivo.

### Datos demográficos de los usuarios

El modelo anterior entonces incorporó  similitud entre las calificaciones de los usuarios para mejorar las recomendaciones. ¿Qué sucede entonces si utilizamos variables demográficas? La intuición es que individuos de similares géneros, profesiones, edades, etc., tendrán preferencias más similares que individuos con características demográficas diferentes.

En este caso lo que haremos es restringir a los ratings de individuos con las mismas características demográficas, en jerga estadística, utilizaremos un modelo de medias condicionales.

Para ello, primero unimos los datos originales sobre usuarios con nuestra base de entrenamiento.

In [19]:
merged_df = pd.merge(X_train, users)

In [20]:
merged_df.head()

,user_id,movie_id,rating,edad,genero,ocupacion,codigo_postal
0,862,177,4,25,M,executive,13820
1,70,193,4,27,M,engineer,60067
2,666,527,4,44,M,administrator,61820
3,535,168,5,45,F,educator,80302
4,603,1240,5,21,M,programmer,47905


Con esta base que tiene ahora variables demográficas podemos intentar mejorar nuestro recomendador. Ilustremos entonces una función recomendadora basada en género y ocupación. Para ello, primero calculamos el rating promedio por género y ocupación. En `pandas` esto es relativamente sencillo utilizando la función `groupby`.

In [21]:
users = users.set_index('user_id')

In [22]:
# Calculamos el rating promedio basado en género y ocupación
gen_occ_mean = merged_df[['genero', 'rating', 'movie_id', 'ocupacion']].pivot_table(
    values='rating', index='movie_id', columns=['ocupacion', 'genero'], aggfunc='mean')

gen_occ_mean.head()

ocupacion administrator           artist              doctor educator  \
genero                F         M      F         M         M        F   
movie_id                                                                
1              3.937500  3.750000    5.0  3.400000  3.666667     3.25   
2              3.000000  3.666667    NaN       NaN       NaN     4.00   
3              3.500000  4.000000    NaN       NaN       NaN      NaN   
4              3.666667  3.600000    NaN  4.666667  3.000000     2.50   
5              4.000000  2.333333    NaN       NaN       NaN     4.00   

ocupacion           engineer           entertainment  ... salesman       \
genero            M        F         M             F  ...        F    M   
movie_id                                              ...                 
1          3.884615      4.0  4.083333           4.0  ...      NaN  4.0   
2          3.500000      NaN  3.066667           NaN  ...      NaN  NaN   
3          2.000000      NaN  3.777778           NaN  ...      NaN  NaN   
4          3.800000      4.0  3.650000           NaN  ...      4.0  4.0   
5          2.333333      NaN  3.500000           NaN  ...      NaN  NaN   

ocupacion scientist        student           technician           writer  \
genero            F    M         F         M          F         M      F   
movie_id                                                                   
1               3.5  4.0  4.043478  3.796296        4.0  3.750000   4.00   
2               NaN  3.0  2.666667  3.277778        NaN  2.714286    NaN   
3               NaN  NaN  3.000000  3.391304        NaN  4.250000    NaN   
4               NaN  3.4  3.250000  3.777778        NaN  3.333333   4.25   
5               NaN  4.0  4.333333  3.111111        NaN  3.333333   4.00   

ocupacion            
genero            M  
movie_id             
1          3.000000  
2          2.333333  
3          1.000000  
4          3.250000  
5          2.000000  

[5 rows x 41 columns]

Tenemos entonces el promedio de rating por género y ocupación. Así, los administradores femeninos calificaron la película 1 con un promedio de 3.93 y los masculinos con 3.75. Mientras que los artistas no calificaron la película 2. Con esta información generamos una función recomendadora que incorpore esta información y en los casos donde no tenemos información utilizaremos el rating 3 como default.

In [23]:
def cf_gen_occ(user_id, movie_id):

    # Verificamos si la película existe en gen_occ_mean
    if movie_id in gen_occ_mean.index:

        #Identificamos el usuario
        user = users.loc[user_id]

        #Identificamos el género y la ocupación
        gender = user['genero']
        occ = user['ocupacion']

        # Verificamos si la ocupación calificó la película
        if occ in gen_occ_mean.loc[movie_id]:

            # Verificamos si el género calificó la película
            if gender in gen_occ_mean.loc[movie_id][occ]:

                # Obtenemos la calificación
                rating = gen_occ_mean.loc[movie_id][occ][gender]

                # Default de 3.0 si no tiene calificación
                if np.isnan(rating):
                    rating = 3.0

                return rating

    # Default de 3.0 si no tiene calificación
    return 3.0

Evaluamos entonces esta función recomendadora:

In [24]:
rmse_recom(cf_gen_occ)

np.float64(1.1419651376788005)

Vemos que esta función es la que peor funciona de las antes evaluadas. La función que incorporaba el coseno como medida de similitud lograba mejores recomendaciones. En la sección siguiente nos centraremos en entender mejor las medidas de similitud y cómo estas nos permiten mejorar los recomendadores.

## Filtrado colaborativo basado en *embeddings*

Como vimos anteriormente parte del problema reside en encontrar productos similares para recomendar. Una forma de lograrlo es utilizando *embeddings*.

Los *embeddings* (también conocidos como incrustaciones en español) son representaciones numéricas de alta dimensionalidad que se utilizan para codificar conceptos, objetos o entidades en un espacio vectorial. Estas representaciones son aprendidas automáticamente a partir de datos y se caracterizan por capturar características y relaciones entre los elementos. Los embeddings son ampliamente utilizados en diversas áreas de la inteligencia artificial, como el procesamiento del lenguaje natural (PLN), la visión por computadora y los sistemas de recomendación.

Un *embedding* o incrustación implica entonces un mapeo de un conjunto discreto (el conjunto de elementos a recomendar) a un espacio vectorial de menor dimensión llamado *embedding space* o *espacio de incrustación*. Este  *embedding space* es de dimensión considerablemente menor al tamaño del espacio original y captura alguna estructura latente de los elementos.

Por ejemplo, supongamos que tenemos una lista de números: [1, 2, 3, 4]. Si queremos representar estos números en un espacio de embedding de 1 dimensión, podríamos usar la siguiente codificación:

    - 1 -> 2.5
    - 2 -> 4.0
    - 3 -> 5.5
    - 4 -> 7.0

En este caso, cada número se ha mapeado a un solo valor en *embedding space* de una dimensión.  Es importante tener en cuenta que, en la práctica, los espacios de embedding suelen tener más dimensiones para capturar mejor la complejidad y las relaciones entre los elementos.


### Ejemplo: Embeddings para recomendar películas

Estudiemos un ejemplo en el que queremos generar recomendaciones de películas a partir de *embeddings*. Supondremos que nuestra plataforma de streaming: **"SlowFlow"** tiene 4 usuarios y 5 películas: [Batman: El caballero de la noche asciende](https://es.wikipedia.org/wiki/The_Dark_Knight_Rises), [Harry Potter y la Piedra Filosofal](https://es.wikipedia.org/wiki/Harry_Potter_y_la_piedra_filosofal), [Shrek](https://es.wikipedia.org/wiki/Shrek), [Las trillizas de Belleville](https://es.wikipedia.org/wiki/Les_Triplettes_de_Belleville) y [Memento](https://es.wikipedia.org/wiki/Memento).

La siguiente matriz que llamaremos $A$, donde $A\in\mathbb{R}^{ m\times n}$, es decir, las filas son los $m$ usuarios y las columnas las $n$ películas, muestra con un $\checkmark$ las películas que cada usuario vió:


![Clase aprendizaje no supervisado](https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/S5_sistemas_recomendacion_I/main/figs/colab0.png)

El objetivo final del ejemplo será generar recomendaciones de películas a nuestros usuarios que no hayan visto.


#### Embedding en una sola dimensión

Comencemos representando las películas y los usuarios a un *embedding space*  de una sola dimensión. Por ejemplo, a lo largo del segmento $[-1,1]$ de forma tal que películas e individuos similares estén más cerca.

Uno de las posibles representaciones que podemos hacer es asignar un escalar que describa la película a lo largo de la dimensión: niños $\rightarrow$ adultos. El escalar va a asignar valores negativos si la película es para niños y positivos si es para adultos. Es decir, estamos resumiendo toda la película a una sola dimensión.  La figura a continuación muestra este posible mapeo:

![Clase aprendizaje no supervisado](https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/S5_sistemas_recomendacion_I/main/figs/colab1.png)

Mapeamos:

- Shrek -> -1
- Las trillizas de Belleville -> -0.8
- Harry Potter -> -0.2
- Batman -> 0.9
- Memento -> 1

Entonces Shrek que es para niños fue mapeada a -1, mientras que  Memento, que es para adultos, a  1. El resto de las películas fueron mapeadas a lo largo de esta dimensión.

Por otro lado, podemos mapear a los 4 individuos a la dimensión de  preferencias por películas para niños o para adultos. Así por ejemplo, podríamos tener el siguiente mapeo:

![Clase aprendizaje no supervisado](https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/S5_sistemas_recomendacion_I/main/figs/colab2b.png)

donde:

1. Amarillo -> -1
2. Azul -> 0
3. Rojo -> 0.1
4. Verde -> 1

Es decir, el individuo amarillo que prefiere películas de niños fue mapeado a 1 películas de niños, mientras que el individuo verde a 1 porque prefiere películas de adultos.

Esta dimensión es útil ya que nos permitiría recomendar películas de niños a aquellos individuos que sólo ven películas de niños, o películas de adultos para aquellos que sólo ven este tipo de películas. Sin embargo hay otras dimensiones que no están siendo capturadas y que potencialmente ayudarían a mejorar las recomendaciones. Continuando con el ejemplo, agreguemos una segunda dimensión.

#### Embeddings en dos dimensiones

Agreguemos ahora otra dimensión: la popularidad de película. Mediremos esta como el grado en que cada película es [cine de culto](https://es.wikipedia.org/wiki/Pel%C3%ADcula_de_culto) o es un éxito de taquilla. En este caso el escalar nuevamente estará entre $[-1,1]$ y tomará -1 si es una película de culto y 1 si es un éxito de taquilla.

Con esta segunda característica, podemos ubicar las películas en un espacio bidimensional:

![Clase aprendizaje no supervisado](https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/S5_sistemas_recomendacion_I/main/figs/colab4.png)

Shrek que fue un éxito de taquilla y una película de niños se ubica en el cuadrante superior izquierdo, mientras que Memento, una película de culto y para adultos esta en el cuadrante inferior derecho. En este caso, cada película se ha mapeado a un vector de 2 dimensiones en el *embedding space*. Las películas con contextos similares tienden a tener embeddings más cercanos entre sí, lo que facilita la identificación de similitudes en el espacio vectorial.

De manera más general, lo que estamos haciendo es reducir la dimensión de las películas a un *embedding space* . En este espacio cada película se describe mediante un conjunto de coordenadas. Por ejemplo, en este espacio, "Shrek" tiene las coordenadas  (-1, 0,95) y "Memento" (1, -0,5). En general, cuando mapeamos a un espacio $d-dimensional$, cada película se representa mediante $d$ números con valores reales, cada uno de los cuales da la coordenada en una dimensión.

De la misma forma, podemos mapear a los individuos en este espacio:

![Clase aprendizaje no supervisado](https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/S5_sistemas_recomendacion_I/main/figs/colab5.png)

Podemos ver la información de los *embeddings* que obtuvimos en los pasos anteriores en la siguiente matriz:

![Clase aprendizaje no supervisado](https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/S5_sistemas_recomendacion_I/main/figs/colab3b.png)

Donde las filas representan a los individuos y el mapeo a las distintos dimensiones dependiendo sus preferencias. Las columnas mientras tanto representan a los películas y el mapeo a las distintas dimensiones.

En estos ejemplos, diseñamos a mano los *embeddings* y nombramos estas dimensiones. En la práctica, los *embeddings*  son aprendidas de los datos lo que lleva a que a menudo las dimensiones individuales no tienen un nombre o interpretación clara.

#### Recomendaciones a partir de Embeddings

En los pasos anteriores construimos dos  *embeddings spaces* uno para las películas y otro para los usuarios. Una ventaja de estos espacios es que nos van a permitir construir recomendaciones.

Con los *embeddings* de los productos podemos construir una matriz de *embeddings* de elementos $V\in\mathbb{R}^{ n\times d}$, donde la fila $j$ es el *embedding* del elemento $j$, que en nuestro ejemplo estará dada por la matriz de *embeddings* de las películas construidas en el paso anterior

 $$
V=\left(\begin{array}{cc}
0.9 & -0.2\\
-1 & -0.8\\
1 & -1\\
1 & 0.9\\
-0.9 & 1
\end{array}\right)
$$

donde la primer columna es la dimensión *culto-> taquillera* y el segundo *niños -> adultos*. De la misma forma tendremos una matriz de  *embeddings* de usuarios $U\in\mathbb{R}^{ m\times d}$ donde la fila $i$ es el *embedding* del usuario $i$:

$$
U=\left(\begin{array}{cc}
1 & 0.1\\
-1 & 0\\
0.2 & -1\\
0.1 & 1
\end{array}\right)
$$

Con estos *embeddings* vamos a buscar  reconstruir la matriz que contiene la información sobre si los usuarios vieron o no la película, la matriz $A\in\mathbb{R}^{ m\times n}$

$$
A=\left(\begin{array}{ccccc}
1 & 0 & 1 & 1 & 0\\
0 & 1 & 0 & 0 & 1\\
1 & 1 & 1 & 0 & 0\\
0 & 0 & 0 & 1 & 1\\
\end{array}\right)
$$

donde 1 indica que el usuario vio la película, y 0 que no. Lo que vamos a hacer aproximar $A$ por el producto de los embeddings $UV^T$.

Haciendo la operación en Python

In [25]:
import numpy as np
U = np.array([[1,0.1], [-1,0], [0.2,-1],[0.1,1]])
VT = np.transpose(np.array([[0.9,-0.2],[-1,-0.8],[1,-1],[1,0.9],[-0.9,1]]))
np.dot(U,VT)

array([[ 0.88, -1.08,  0.9 ,  1.09, -0.8 ],
       [-0.9 ,  1.  , -1.  , -1.  ,  0.9 ],
       [ 0.38,  0.6 ,  1.2 , -0.7 , -1.18],
       [-0.11, -0.9 , -0.9 ,  1.  ,  0.91]])

Tenemos entonces una aproximación a la matriz $A$ pero también obtenemos información adicional en las entradas que antes tenían cero. Esta información puede ser utilizada para generar las recomendaciones. Asi por ejemplo, al cuarto usuario primero le recomendaríamos Shrek, mientras que al segundo usuario le recomendaríamos Batman.

![Clase aprendizaje no supervisado](https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/S5_sistemas_recomendacion_I/main/figs/colab7c.png)

### Generación de *Embeddings*

Finalmente queda discutir como generar estas matrices de *embeddings*. La sección anterior nos da una idea de como hacerla, dada la matriz observada $A\in\mathbb{R}^{ m\times n}$ con $m$ usuarios y $n$ ítems, queremos encontrar una representación de esta matriz en una dimensión menor y que capture las relaciones subyacentes entre usuarios y películas.

En la semana 2 del curso estudiamos la Descomposición en Valores Singulares (SVD) como técnica de reducción de dimensión. Recordemos que esta técnica descompone la matriz original en tres:

$$
A = US V'
$$

En nuestro caso vamos a descomponer la matriz de ratings $A$, que describe las interacciones usuario-películas, en tres matrices de menor dimensión:

- $U$  la matriz de *embeddings* de usuarios,
- $V$ la  matriz de *embeddings* de películas,
- $S$ la matriz diagonal de los valores singulares.

La matriz diagonal de valores singulares ($S$) es fundamental para comprender la estructura latente que se captura durante el proceso de SVD en sistemas de recomendación. Estos valores singulares están ordenados de mayor a menor, lo que nos permite truncar la descomposición para conservar solo los primeros $d$ valores singulares más importantes reduciendo efectivamente la dimensión. Estos $d$ valores singulares determinarán también la dimensión del *embedding space*.

La estructura latente capturada por los valores singulares se refiere a las relaciones y patrones subyacentes en los datos originales que son descubiertos por la SVD. Cada valor singular en la matriz $S$ representa la importancia de una dimensión específica en la representación de la matriz original. Al estar ordenados de mayor a menor, significa que los primeros valores singulares capturan la mayor cantidad de información relevante, mientras que los valores singulares más pequeños representan información menos relevante o ruido.

Cuando seleccionamos solo los primeros $d$ valores singulares más importantes para reconstruir la matriz original, estamos reduciendo la dimensionalidad de los datos. La dimensionalidad reducida se refiere a que solo estamos considerando las $d$ dimensiones más significativas para representar los usuarios y películas en un espacio de *embeddings*

Esta reducción de dimensionalidad permite que el modelo capture de manera más eficiente las relaciones y similitudes entre usuarios y películas, ya que las dimensiones menos relevantes o ruidosas se descartan. Esencialmente, los *embeddings* resultantes de SVD se encuentran en un espacio de menor dimensión, pero aún conservan una representación significativa de los patrones de interacción entre usuarios y películas. Es por ello que la selección del número adecuado de valores singulares es un paso crítico en la aplicación de SVD para sistemas de recomendación. Un valor demasiado pequeño puede resultar en una pérdida significativa de información y, por lo tanto, en recomendaciones inexactas. Por otro lado, un valor demasiado grande puede llevar a problemas de sobreajuste y un aumento en la complejidad computacional.

La estructura latente capturada por los valores singulares se puede interpretar como características o conceptos latentes que subyacen a las interacciones entre usuarios y películas. Estas características pueden representar preferencias de los usuarios, intereses específicos o temáticas de los películas. En nuestro ejemplo las características latentes eran si la película era para niños o adultos y su éxito en la taquilla. Pero podríamos tener otras  características latentes que en ocasiones, podemos inspeccionar y asignarle una interpretación. Sin embargo, generalmente esto no será posible ya que estarán capturando una dimensión subyacente, ya que no representa una característica no explícita en los datos sino que se deduce de ellos.

Al capturar estas características latentes, SVD permite una representación más compacta y significativa de los usuarios y las películas, lo que facilita la recomendación eficiente de películas similares a los que el usuario ha mostrado interés previamente.


En resumen, los valores singulares en la SVD nos permiten identificar y representar las características latentes más importantes en los datos originales, lo que facilita la recomendación precisa y eficiente de elementos para los sistemas de recomendación basados en usuarios.


### Implementación en `Python`

Implementemos esta estrategia en `Python`.  Notemos que a diferencia del ejemplo anterior, la matriz del ejemplo que venimos utilizando en `Python` que vincula usuarios, películas y ratings, contiene ratings para las películas vistas y calificadas por el usuario y faltantes (`NaN`) en caso contrario.

Para poder implementar SVD primero necesitamos transformar los valores no observados de la matriz de ratings en 0.

In [26]:
A_df = ratings.pivot(index = 'user_id', columns ='movie_id', values = 'rating').fillna(0)
A_df.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Tenemos entonces un `DataFrame` con los usuarios en las filas y las películas en la columnas. Necesitamos ahora transformarlas a una matriz y centrar en cero utilizando la media de los ratings de cada usuario para aplicar SVD.

In [27]:
A = A_df.to_numpy()
user_ratings_mean = np.mean(A, axis = 1)
A_demeaned = A - user_ratings_mean.reshape(-1, 1)

Estamos ahora en posición de realizar la SVD. En el *cuaderno: Descomposición en Valores Singulares. Fundamentos Teóricos* utilizamos [Numpy](https://numpy.org/) en nuestros ejemplos, en este cuaderno usaremos [SciPy](https://scipy.org/) y los invito a que prueben utilizando [Numpy](https://numpy.org/).

La ventaja de [SciPy](https://scipy.org/) es que permite elegir los valores singulares como argumento de la función en vez de tener que truncar la matriz posteriormente.

En este ejemplo, utilizaremos 50 para ilustrar.

In [28]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(A_demeaned, k = 50)

Para realizar nuestra recomendación tenemos que reconstruir la matriz $A$ con esta aproximación menor. Es decir tendremos que multiplicar $U$, $S$, and $V^{T}$ para obtener la aproximación de rango $d=50$ de $A$. Puesto que centramos en cero los ratings, necesitamos agregarlos nuevamente.

Haciendo esto tenemos:


In [29]:
sigma = np.diag(sigma)

A_recomendaciones = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

Notemos que elegimos $d=50$ de forma arbitraria. Si queremos optimizar este recomendador, tendríamos que proceder de forma similar a cómo lo hicimos  en el en el *cuaderno: Descomposición en Valores Singulares. Fundamentos Teóricos*  y estudiar el comportamiento de los valores singulares. En los sistemas de recomendación, una caída rápida de los valores singulares indica que hay patrones dominantes en los datos que pueden ser capturados por unos pocos valores singulares. Estos patrones dominantes pueden ser utilizados para hacer recomendaciones.

O también buscar un enfoque supervisado donde dividimos la muestra en entrenamiento y prueba, y se busca el número $d$ que minimize el RMSE.

Los invito a que prueben ambas estrategias por su cuenta y comparen los resultados con los recomendadores que construimos anteriormente.

##### Recomendando películas.

Con la matriz de recomendaciones podemos construir una función que genere recomendaciones a cualquier usuario.

Esta función generará recomendaciones de películas para un usuario específico de forma tal que omita a aquellas que ya ha visto/calificado. Estamos asumiendo que si la calificó es porque ya la vió, pero como comparación retornaremos por separado aquellas películas ya calificadas.

Comenzamos poniendo las recomendaciones en un DataFrame:

In [30]:
recomendac_df = pd.DataFrame(A_recomendaciones, columns = r_matrix.columns)
recomendac_df.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,6.488436,2.959503,1.634987,3.024467,1.656526,1.659506,3.630469,0.240669,1.791518,3.347816,...,0.011976,-0.092017,-0.074553,-0.060985,0.009427,-0.035641,-0.039227,-0.037434,-0.025552,0.023513
1,2.347262,0.129689,-0.098917,0.328828,0.159517,0.481361,0.213002,0.097908,1.892100,0.671000,...,0.003943,-0.026939,-0.035460,-0.029883,-0.027153,-0.015244,-0.008277,-0.011760,0.011639,-0.046924
2,0.291905,-0.263830,-0.151454,-0.179289,0.013462,-0.088309,-0.057624,0.568764,-0.018506,0.280742,...,-0.028964,-0.031622,0.045513,0.026089,-0.021705,0.002282,0.032363,0.017322,-0.006644,-0.009480
3,0.366410,-0.443535,0.041151,-0.007616,0.055373,-0.080352,0.299015,-0.010882,-0.160888,-0.118834,...,0.020069,0.015981,-0.000182,0.005593,0.026634,0.023562,0.036405,0.029984,0.015612,-0.008713
4,4.263488,1.937122,0.052529,1.049350,0.652765,0.002836,1.730461,0.870584,0.341027,0.569055,...,0.019973,-0.053521,-0.017242,-0.007137,-0.038987,0.010338,0.004869,0.007603,-0.020575,0.003330


Con este DataFrame armamos nuestra función recomendadora. Esta función tomara como argumentos la base que construimos de recomendaciones predichas, el número de usuario, la base original de películas y el número de películas que queremos que recomiende. La función entonces buscará el usuario para el que queremos generar una recomendación, luego buscará las películas que ha calificado, para luego generar un numero de películas recomendadas a partir de las películas que no ha visto y el rating que se generó al reconstruir  la matriz $A$:

In [31]:
def recomendador(predictions_df, user_id, movies_df, original_ratings_df, num_recommendations=5):

    # Obtenemos y ordenamos los usuarios
    user_row_number = user_id - 1 # user_id inicia en 1, no 0, corregimos el índice
    sorted_user_predictions = recomendac_df.iloc[user_row_number].sort_values(ascending=False) # UserID starts at 1

    # Obtenemos los datos de los usuarios y le agremamos los datos de la película
    user_data = original_ratings_df[original_ratings_df.user_id == (user_id)]
    user_full = (user_data.merge(movies, how = 'left', left_on = 'movie_id', right_on = 'movie_id').
                     sort_values(['rating'], ascending=False)
                 )

    print('El usuario {0} ha calificado {1} películas.'.format(user_id, user_full.shape[0]))
    print('Recomendando las {0} películas que no han sido calificadas.'.format(num_recommendations))

    # Generamos la recomendación de las películas que no ha visto todavía
    recomendaciones = (movies_df[~movies_df['movie_id'].isin(user_full['movie_id'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movie_id',
               right_on = 'movie_id').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recomendaciones

Notemos que agregamos la base de películas a la función para poder incluir características explícitas de las películas, aunque no las utilizamos en la generación de los *embeddings* y por lo tanto en la recomendación.

In [32]:
calificadas, recomendaciones = recomendador(recomendac_df, 837, movies, ratings, 10)

El usuario 837 ha calificado 46 películas.
Recomendando las 10 películas que no han sido calificadas.


Veamos entonces las películas que ha visto y las recomendaciones generadas. El usuario 837, ha calificado 46 películas, entre las que figuran películas de romance com 'Jane Eyre' e infantiles como 'Willy Wonka and the Chocolate Factory':

In [33]:
calificadas.head(10)

,user_id,movie_id,rating,titulo
11,837,289,5,Evita (1996)
9,837,283,5,Emma (1996)
36,837,125,5,Phenomenon (1996)
37,837,1009,5,Stealing Beauty (1996)
44,837,740,5,Jane Eyre (1996)
24,837,151,5,Willy Wonka and the Chocolate Factory (1971)
6,837,19,4,Antonia's Line (1995)
19,837,286,4,"English Patient, The (1996)"
14,837,20,4,Angels and Insects (1995)
27,837,328,4,Conspiracy Theory (1997)


y las recomendaciones que hicimos fueron:

In [34]:
recomendaciones

,movie_id,titulo
11,14,"Postino, Il (1994)"
0,1,Toy Story (1995)
107,116,Cold Comfort Farm (1995)
116,126,"Spitfire Grill, The (1996)"
42,50,Star Wars (1977)
238,255,My Best Friend's Wedding (1997)
440,471,Courage Under Fire (1996)
92,100,Fargo (1996)
258,282,"Time to Kill, A (1996)"
704,742,Ransom (1996)


En estas recomendaciones aparecen películas de romance como "Il Postino" e infantiles como "Toy Story". Estas parecen ser recomendaciones bastante buenas. Notemos además que a pesar de que no incluimos el género como característica en la factorización, esta fue capaz de captar el *embedding* con esta dimensión.

## Consideraciones finales

En este *cuaderno* estudiamos distintos formas de construir filtrados colaborativos comenzando con formas sencillas de incorporar información de otros usuarios hasta capturar características latentes con factorización de matrices.

Sin embargo, al factorizar las matrices y reducir su dimensión hay que tener en cuenta que es probable que se esté perdiendo parte de las señales. También que considerar que cuando las matrices son poco densas, es decir, las entradas estan faltantes, la solución de SVD estará cercana a cero, produciendo recomendaciones poco eficientes.


En casos como estos se suele utilizar  **factorización matricial ponderada**:

$$
\min_{U\in\mathbb{R}^{m\times d}, V\in\mathbb{R}^{n\times d}} \sum_{(i,j)\in obs}(A_{i,j} - \langle U_i, V_j \rangle)^2 + \omega_0 \sum_{(i,j)\not\in obs} (\langle U_i, V_j \rangle)^2
$$

Esta descompone la función objetivo en dos sumas:

- Una suma sobre las entradas observadas.
- Una suma sobre entradas no observadas (tratadas como ceros).

donde, $\omega_0$ es un hiperparámetro que pondera los dos términos para que la función objetivo no esté dominada por uno u otro componente. En estos casos, será clave ajustar correctamente este hiperparámetro.

En ocaciones, cuando hay  ítems muy frecuentes o populares que pueden dominar la función objetivo se suele ponderar por la frecuencia de los elementos. En otras palabras, podemos reemplazar la función objetivo por:

$$\sum_{(i,j)\in obs}\omega_{i,j} (A_{i,j} - \langle U_i, V_j \rangle)^2 + \omega_0 \sum_{(i,j)\not\in obs} (\langle U_i, V_j \rangle)^2$$

Donde $\omega_{i,j}$ es una función de la frecuencia de ocurrencia.

Es importante también destacar que SVD es una técnica lineal y, aunque es efectiva para capturar estructuras latentes en muchos casos, también existen técnicas no lineales más avanzadas, como factorización de matrices no negativas (NMF) o *embeddings* basados en redes neuronales, que pueden capturar relaciones más complejas y no lineales en los datos pero que están mas allá de este curso.



# Referencias

- Banik, R. (2018). Hands-on recommendation systems with Python: start building powerful and personalized, recommendation engines with Python. Packt Publishing Ltd.

- Covington, P., Adams, J., & Sargin, E. (2016). Deep Neural Networks for YouTube recommendations. Proceedings of the 10th ACM Conference on Recommender Systems. https://doi.org/10.1145/2959100.2959190

- Google developers. (n.d.). Recommendation systems. Google. Consultado en Abril 3, 2022. Disponible en https://developers.google.com/machine-learning/recommendation/overview

- Google developers. (n.d.). Embeddings: Motivation From Collaborative Filtering. Consultado en Mayo 13, 2022. Disponible en  https://developers.google.com/machine-learning/crash-course/embeddings/motivation-from-collaborative-filtering
